# Data collection of spanish vocabulary

## Objective :
- Establish Request connect using Requests Lib
- Convert web page to html script with Beautiful
- Collect Data in containers
- Using DataFrame to refonction Data
- Save Dataframes in csv files


In [46]:
# import required lib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os

## Finding the Main page


In [2]:
url = 'https://www.ver-taal.com/vocabulario.htm'

In [3]:
# accessing the html content form webpage
response = requests.get(url)

# parsing the html content
soup = BeautifulSoup(response.content, 'html')

# get the title 
title = soup.title.text + '.csv'

title           

'Ejercicios de vocabulario.csv'

In [4]:
# extract the element from list that we will include it as headers

headers_contenaire =[] # here we where store our hearders

# this tag contains the list of elements
headers = soup.select_one('ul', attri={'class':'nolist'})

type(headers)

bs4.element.Tag

In [5]:
# Iterate all row in list in html
for row in headers:
    
    # romove the element which
    if row != '\n':
    
        headers_contenaire.append(row.text) # add each row to the list

print(headers_contenaire)

['La casa\nEl cuerpo y la salud\nLos animales\nLa comida\nLa ropa\nLa familia\nEl transporte\nEl ocio\nEl trabajo\nEl mundo vegetal\nEl Mundo\nLa ciudad\nLos colores, los números y las formas\n']


In [6]:
# Remove Empty, underline elements
headers_contenaire = list(filter(None, headers_contenaire[0].split('\n')))

headers_contenaire

['La casa',
 'El cuerpo y la salud',
 'Los animales',
 'La comida',
 'La ropa',
 'La familia',
 'El transporte',
 'El ocio',
 'El trabajo',
 'El mundo vegetal',
 'El Mundo',
 'La ciudad',
 'Los colores, los números y las formas']

# Extract vocabulary from html page

In [7]:
# let find tag that includs all urls
a = headers.find_all('a')


#  titles of groups of vacabularies
containers_titles = []

#  titles without contents
nones = []

# parant container that collect subcluster
cluster =[]
cluster_title = []

# iterate the url form the main page 
for href in a:
    # child container groups of vocabularies according to their term
    subcluster =[]
    subcluster_title = []
    
    # Extract all links of rows
    url = 'https://www.ver-taal.com/' + href.get('href')

    # parsing each url using BeautifulSoup
    url2 = requests.get(url)

    soup_2 = BeautifulSoup(url2.content, 'html.parser') # convert url to html script

    table = soup_2.table 

    # looping the child contents 
    for row in table.find_all('tr'):
        

        rows = row.find('a')

        # choose only existing rows
        if rows is not None:

            rurl = 'https://www.ver-taal.com/' + rows.get('href')
        

            res2 = requests.get(rurl)

            soup_3 = BeautifulSoup(res2.content, "html.parser")
                        
            containers_titles.append(soup_3.title.text)

            elements = soup_3.find('select')

            # Check if elements is not None before iterating
            if elements is not None:
                
                # collect voc from the same child
                label = []

                # iterate the vocabularies from  each url 
                for i in elements:

                    if i.text is not None and i.text != '\n' and i.text != '--seleccionar--' :

                        label.append(i.text)

            else:
                
                nones.append(soup_3.title.text) 

        # collect the child groups and their titles
        subcluster.append(label)

        subcluster_title.append(soup_3.title.text)

    # collect the subclusters and their titles
    cluster.append(subcluster)

    cluster_title.append(subcluster_title)
              
        

In [31]:
cluster_title[1]

['Ejercicio de vocabulario: las partes del cuerpo (1)',
 'Ejercicio de vocabulario: las partes del cuerpo (1)',
 'Ejercicio de vocabulario: las partes del cuerpo (1)',
 'Vocabulario: partes del cuerpo (2)',
 'Vocabulario: partes del cuerpo (3)',
 'Vocabulario: la cabeza (1)',
 'Vocabulario: la cabeza (2)',
 'Vocabulario: la cabeza (3)',
 'Vocabulario: la mano y el pie',
 'Vocabulario: los órganos',
 'Vocabulario: los huesos',
 'Vocabulario: los dientes',
 'Vocabulario: fluidos corporales',
 'Vocabulario: manchas de la piel y problemas cutáneos',
 'Vocabulario: sensaciones corporales',
 'Vocabulario: posturas corporales',
 'Vocabulario: Descripción de una persona',
 'Vocabulario: Descripción de una persona',
 'Vocabulario: Descripción de una persona',
 'Vocabulario: el pelo (1)',
 'Vocabulario: el pelo (2)',
 'Vocabulario: la higiene',
 'Vocabulario: la higiene',
 'Vocabulario: la higiene',
 'Vocabulario: la belleza y el maquillaje',
 'Vocabulario: tipos de medicamentos',
 'Vocabulario:

In [8]:
# filter the titles without content
dfcolumns = [word for word in  containers_titles if word not in nones]
len(dfcolumns)


193

In [35]:
# modify titles of cluster

clustertitle = [[word.replace('Vocabulario: ', '') for word in array if 'Vocabulario: '  in word or 'Ejercicio de vocabulario: '  in word] for array in cluster_title]

print(len(clustertitle))

13


## Convert Containers into DataFrame

In [36]:
def table(n):
    print(headers_contenaire[n])
    dff = pd.DataFrame(cluster[n])
    dff = dff.transpose()
    try:

        dff.columns = [clustertitle[n]]
    except:
        dff.columns = [cluster_title[n]]
   

    # Drop duplicate columns
    df_ = dff.T.drop_duplicates().T
    return df_


#index of tables

my_dict = {i: item for i, item in enumerate(headers_contenaire)}

indexTable = pd.DataFrame([headers_contenaire])

indexTable


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,La casa,El cuerpo y la salud,Los animales,La comida,La ropa,La familia,El transporte,El ocio,El trabajo,El mundo vegetal,El Mundo,La ciudad,"Los colores, los números y las formas"


In [38]:
# calling table function 
table(0)

La casa


,partes de la casa,algunos muebles,en la mesa,en el dormitorio,en el cuarto de baño,en la cocina,en el salón-comedor,en el despacho (1),en el despacho (2),en el despacho (3),...,la puerta,suelos y paredes,la electricidad,los utensilios de cocina,electrodomésticos,electrodomésticos (2),en el jardín,el exterior de la casa,utensilios de limpieza,tareas domésticas
0,el cuarto de baño,el armario,la botella,la almohada,la bañera,el cazo,la alfombra,el bolígrafo,la chincheta,la almohadilla,...,la aldaba,el adoquín,la bombilla,el abrebotellas,la aspiradora,la batidora,el banco,la barandilla,la bayeta,cocinar
1,la casa,la cama,la cafetera,el colchón,la báscula,la cacerola,la calefacción,la calculadora,la cinta adhesiva,la bandeja,...,la bisagra,la alfombra,la bombilla de ahorro,el abrelatas,la cafetera,el cortafiambres,la carretilla,el canalón,la escoba,cortar el césped
2,la cocina,la estantería,la copa,el despertador,la ducha,la campana,la cortina,la carpeta,el clip,la goma de borrar,...,el buzón,el azulejo,el cable alargador,el batidor,el congelador,el cuecehuevos,la caseta,la chimenea,la escobilla,cuidar el jardín
3,el comedor,la mesa,la cuchara,la litera,el espejo,la cocina,la chimenea,el escritorio,la grapa,la goma elástica,...,la cerradura,la baldosa,la clavija,el cascanueces,el exprimidor,la freidora,el cubo,la contraventana,la esponja,hacer la compra
4,el despacho,la puerta,el cuchillo,la manta,el jabón,el fregadero,el jarrón de flores,la impresora,la grapadora,el pegamento,...,el cerrojo,el empapelado,el enchufe,el colador,el hervidor,la licuadora,la escalera,la escalera,el estropajo,lavar la ropa
5,el dormitorio,la silla,el plato,la mesita de noche,el lavabo,el grifo,la lámpara,el lápiz,el rotulador,la pinza,...,el felpudo,el entarimado,el fusible,el cucharón,la lavadora,el molinillo de café,el guante,la maceta,la fregona,lavar los platos
6,el garaje,el sillón,la servilleta,la percha,el papel higiénico,el horno,la pecera,el libro,el tablón de anuncios,el tampón,...,la llave,el hormigón,el interruptor,el embudo,el lavavajillas,la olla a presión,la manguera,el pasamanos,el limpiacristales,limpiar
7,el jardín,el sofá,la taza,la persiana,el peine,la nevera,la planta,el ordenador,la taladradora,la tinta,...,el ojo de la cerradura,el mármol,la pila,la espumadera,el microondas,la panificadora,la pala,el peldaño,el paño,pasar la aspiradora
8,el salón,el televisor,el tenedor,el ropero,el retrete,la placa,el reloj,la papelera,None,None,...,el picaporte,la moqueta,la regleta,el rallador,la plancha,la plancha de asar,la regadera,la teja,el plumero,planchar
9,la terraza,la ventana,el vaso,la sábana,la toalla,la sartén,el teléfono,las tijeras,None,None,...,el timbre,la pintura,el tubo fluorescente,el sacacorchos,la tostadora,la sandwichera,la valla,el tejado,el recogedor,tender la ropa


## Convert Dataframe into a Quiz with result

In [15]:
def tableQuiz(n):
    print(headers_contenaire[n])
    dff = pd.DataFrame(cluster[n])
    dff = dff.transpose()
    dff.columns = [cluster_title[n]]
    # Drop duplicate columns
    df_ = dff.T.drop_duplicates().T
    
    # quiz
    name = input('Elige el tema de Vocabularion: ')

    res = []

    if name in df_.columns :
            
        for i in range(len(df_[name].dropna())):
        
            voc = input(f'>{i}  ')

            res.append(voc)  
        
        diff1 = [word2 for word2 in df_[name].dropna() if word2 in res   ] 
        diff3 = [word2 for word2 in df_[name].dropna() if word2 in res  ]  
        
        quiz = df_[name].dropna()

        quiz['Respuestas Proporcianadas'] = res

        try:

            quiz['Respuestas Correctas'] = [word2 if word2 in df_[name].dropna() else 0 for word2 in res  ]  
        except:
            pass

        print("Tu puntaji:",len(diff1), '/', len(df_[name].dropna()))
        print(quiz)
        
     

    else :
        return 'No es incluido'
    

tableQuiz(0)


La casa


'No es incluido'

In [40]:
indexTable = pd.DataFrame([headers_contenaire])

indexTable

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,La casa,El cuerpo y la salud,Los animales,La comida,La ropa,La familia,El transporte,El ocio,El trabajo,El mundo vegetal,El Mundo,La ciudad,"Los colores, los números y las formas"


## Save Dataframes in csv files

In [92]:
def Importcsv(n):
   
    dff = pd.DataFrame(cluster[n])
    dff = dff.transpose()
    try:

        dff.columns = [clustertitle[n]]
    except:
        dff.columns = [cluster_title[n]]
   

    # Drop duplicate columns
    df_ = dff.T.drop_duplicates().T
  

    # Specify the base path where folders should be created
    base_path = 'csv_container'  # Replace with your desired directory
        
            
    file_name = f"{headers_contenaire[n]}.csv"
    file_path = os.path.join(base_path, file_name)

    # Save the DataFrame to the CSV file
    df_.to_csv(file_path, index=False, encoding='UTF-8')
    print(f"Saved {file_path}")
       
   

# Iterate over the headers_contenaire and process each cluster
for i in range(len(headers_contenaire)):
    Importcsv(i)

La casa
Saved csv_container\La casa.csv
El cuerpo y la salud
Saved csv_container\El cuerpo y la salud.csv
Los animales
Saved csv_container\Los animales.csv
La comida
Saved csv_container\La comida.csv
La ropa
Saved csv_container\La ropa.csv
La familia
Saved csv_container\La familia.csv
El transporte
Saved csv_container\El transporte.csv
El ocio
Saved csv_container\El ocio.csv
El trabajo
Saved csv_container\El trabajo.csv
El mundo vegetal
Saved csv_container\El mundo vegetal.csv
El Mundo
Saved csv_container\El Mundo.csv
La ciudad
Saved csv_container\La ciudad.csv
Los colores, los números y las formas
Saved csv_container\Los colores, los números y las formas.csv
